In [52]:
import yaml,logging,numpy,pandas
from optparse import OptionParser
from feature import Feature,languages
from matplotlib import pyplot as plt
%matplotlib inline
features = yaml.load(open('features.yml'))
logging.basicConfig(level=logging.ERROR)

**How Many Languages per Feature**

In [53]:
coverage = languages.ix[:,10:]
coverage = coverage.replace(to_replace=".+",regex=True,value=1)
coverage = coverage.replace(to_replace='',value=0)
feature_cover = coverage.apply(lambda f: sum(f))
feature_cover.describe()

count     192.000000
mean      398.255208
std       349.743066
min         5.000000
25%       171.500000
50%       257.000000
75%       508.250000
max      1519.000000
dtype: float64

**How Many Features per Language**

In [54]:
coverage['features_count'] = coverage.apply(lambda x: sum(x), axis=1)
coverage['features_count'][coverage['features_count'] > 120].describe()

count    107.000000
mean     136.841121
std       10.499910
min      121.000000
25%      127.500000
50%      137.000000
75%      144.000000
max      159.000000
Name: features_count, dtype: float64

So we have 107 languages with more than 120 features.
Let's intesect these rows with the top quartile of features (that is features that cover more than 508 languages):

In [55]:
q = feature_cover.quantile(0.75)
top_features = list()
for i in feature_cover.keys():
    if feature_cover[i] > q:
        top_features.append(i)

top_langs = coverage[coverage['features_count'] > 120]

How many languages over all have all or allmost all the top features in common?

In [56]:
intersection = coverage[coverage['features_count'] > 120].filter(items=top_features)
for i,r in intersection.iterrows():
    if r.sum() > 45:
        print(languages.iloc[i]['Name'])

Abkhaz
Basque
English
Evenki
Finnish
French
Georgian
German
Greek (Modern)
Hausa
Hungarian
Indonesian
Jakaltek
Japanese
Ju|'hoan
Kewa
Koromfe
Khalkha
Kanuri
Korean
Lezgian
Malagasy
Maori
Mangarrayi
Russian
Slave
Spanish
Thai
Tiwi
Turkish
Wichí
Yaqui


In [232]:
tranpose= intersection.transpose()
coverage1 = tranpose.ix[:,2:]
coverage1['languages_count'] = coverage1.apply(lambda x: sum(x), axis=1)
coverage1['languages_count'].describe()

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


count     48.000000
mean      97.916667
std       14.573122
min       53.000000
25%       97.750000
50%      104.500000
75%      105.000000
max      107.000000
Name: languages_count, dtype: float64

In this intersection, take the features that have more than a 100 of the 107 languages

In [234]:
complete = coverage1[coverage1['languages_count'] == 107]
complete['languages_count']

82A Order of Subject and Verb                           107
83A Order of Object and Verb                            107
143F Postverbal Negative Morphemes                      107
143E Preverbal Negative Morphemes                       107
143A Order of Negative Morpheme and Verb                107
143G Minor morphological means of signaling negation    107
Name: languages_count, dtype: int64

** The raw values may be ok for MCA, lets save csv and try in R **

In [292]:
completeraw = languages.filter(items=complete.columns,axis=0)[complete.index]
completeraw[:10]

,82A Order of Subject and Verb,83A Order of Object and Verb,143F Postverbal Negative Morphemes,143E Preverbal Negative Morphemes,143A Order of Negative Morpheme and Verb,143G Minor morphological means of signaling negation
8,1 SV,1 OV,2 [V-Neg],2 [Neg-V],11 Type 3 / Type 4,4 None
34,1 SV,2 VO,2 [V-Neg],3 NegV&[Neg-V],15 OptDoubleNeg,4 None
54,1 SV,1 OV,4 None,1 NegV,1 NegV,4 None
70,1 SV,1 OV,4 None,1 NegV,1 NegV,4 None
89,1 SV,1 OV,2 [V-Neg],1 NegV,15 OptDoubleNeg,4 None
128,1 SV,3 No dominant order,4 None,1 NegV,1 NegV,4 None
140,1 SV,3 No dominant order,4 None,3 NegV&[Neg-V],7 Type 1 / Type 3,4 None
143,1 SV,2 VO,1 VNeg,1 NegV,15 OptDoubleNeg,4 None
153,1 SV,1 OV,1 VNeg,4 None,2 VNeg,4 None
183,1 SV,1 OV,3 VNeg&[V-Neg],1 NegV,14 ObligDoubleNeg,4 None


In [293]:
completeraw.to_csv('raw-complete-107x6.csv')

But Let's also try a looser restriction:

In [247]:
complete2 = coverage1[coverage1['languages_count'] > 100]
complete2raw = languages.filter(items=complete2.columns,axis=0)[complete2.index]
complete2raw[:5]

,1A Consonant Inventories,2A Vowel Quality Inventories,3A Consonant-Vowel Ratio,4A Voicing in Plosives and Fricatives,5A Voicing and Gaps in Plosive Systems,6A Uvular Consonants,7A Glottalized Consonants,8A Lateral Consonants,11A Front Rounded Vowels,13A Tone,...,88A Order of Demonstrative and Noun,89A Order of Numeral and Noun,97A Relationship between the Order of Object and Verb and the Order of Adjective and Noun,112A Negative Morphemes,116A Polar Questions,143F Postverbal Negative Morphemes,"144A Position of Negative Word With Respect to Subject, Object, and Verb",143E Preverbal Negative Morphemes,143A Order of Negative Morpheme and Verb,143G Minor morphological means of signaling negation
8,5 Large,1 Small (2-4),5 High,4 In both plosives and fricatives,2 None missing in /p t k b d g/,2 Uvular stops only,2 Ejectives only,"2 /l/, no obstruent laterals",1 None,1 No tones,...,1 Demonstrative-Noun,3 No dominant order,2 OV and NAdj,1 Negative affix,2 Interrogative verb morphology,2 [V-Neg],20 MorphNeg,2 [Neg-V],11 Type 3 / Type 4,4 None
34,4 Moderately large,2 Average (5-6),4 Moderately high,4 In both plosives and fricatives,3 Missing /p/,4 Uvular stops and continuants,1 No glottalized consonants,"2 /l/, no obstruent laterals",1 None,1 No tones,...,2 Noun-Demonstrative,3 No dominant order,4 VO and NAdj,2 Negative particle,1 Question particle,2 [V-Neg],19 OptDoubleNeg,3 NegV&[Neg-V],15 OptDoubleNeg,4 None
54,1 Small,2 Average (5-6),2 Moderately low,1 No voicing contrast,1 Other,1 None,1 No glottalized consonants,1 No laterals,1 None,2 Simple tone system,...,1 Demonstrative-Noun,1 Numeral-Noun,1 OV and AdjN,2 Negative particle,1 Question particle,4 None,7 SONegV,1 NegV,1 NegV,4 None
70,2 Moderately small,3 Large (7-14),2 Moderately low,2 In plosives alone,2 None missing in /p t k b d g/,1 None,1 No glottalized consonants,1 No laterals,1 None,1 No tones,...,1 Demonstrative-Noun,1 Numeral-Noun,1 OV and AdjN,2 Negative particle,6 Interrogative intonation only,4 None,7 SONegV,1 NegV,1 NegV,4 None
89,2 Moderately small,2 Average (5-6),3 Average,2 In plosives alone,3 Missing /p/,1 None,1 No glottalized consonants,"2 /l/, no obstruent laterals",1 None,1 No tones,...,2 Noun-Demonstrative,2 Noun-Numeral,2 OV and NAdj,2 Negative particle,1 Question particle,2 [V-Neg],19 OptDoubleNeg,1 NegV,15 OptDoubleNeg,4 None


33 is too much for 108 languages. Let's Extract The most varied features. But First let's see how bad it is

In [260]:
for lang in complete2raw.iterrows():
    ok = True
    for i,feature_value in enumerate(lang[1]):
        if feature_value == "":
            print(languages.iloc[lang[0]]['Name'],"missing",complete2raw.columns[i])
            break

Chamorro missing 51A Position of Case Affixes
Canela-Krahô missing 89A Order of Numeral and Noun
Fijian missing 33A Coding of Nominal Plurality
Gooniyandi missing 116A Polar Questions
Hebrew (Modern) missing 1A Consonant Inventories
Hixkaryana missing 88A Order of Demonstrative and Noun
Hmong Njua missing 33A Coding of Nominal Plurality
Imonda missing 26A Prefixing vs. Suffixing in Inflectional Morphology
Kayardild missing 144A Position of Negative Word With Respect to Subject, Object, and Verb
Ket missing 112A Negative Morphemes
Kiowa missing 87A Order of Adjective and Noun
Koasati missing 69A Position of Tense-Aspect Affixes
Krongo missing 85A Order of Adposition and Noun Phrase
Lavukaleve missing 88A Order of Demonstrative and Noun
Luvale missing 51A Position of Case Affixes
Meithei missing 33A Coding of Nominal Plurality
Ngiyambaa missing 1A Consonant Inventories
Oneida missing 87A Order of Adjective and Noun
Paiwan missing 33A Coding of Nominal Plurality
Quechua (Imbabura) missing

If we just take the first numeric value of the cell, the variance of a column can provide a rough estimate of the variance in this feature among the 108 languages.

In [267]:
from extractors import numerize
logging.basicConfig(level=logging.WARNING)
numerized = complete2raw.applymap(lambda x : numerize(x) if x != "" else 0)
numerized[:5]

,1A Consonant Inventories,2A Vowel Quality Inventories,3A Consonant-Vowel Ratio,4A Voicing in Plosives and Fricatives,5A Voicing and Gaps in Plosive Systems,6A Uvular Consonants,7A Glottalized Consonants,8A Lateral Consonants,11A Front Rounded Vowels,13A Tone,...,88A Order of Demonstrative and Noun,89A Order of Numeral and Noun,97A Relationship between the Order of Object and Verb and the Order of Adjective and Noun,112A Negative Morphemes,116A Polar Questions,143F Postverbal Negative Morphemes,"144A Position of Negative Word With Respect to Subject, Object, and Verb",143E Preverbal Negative Morphemes,143A Order of Negative Morpheme and Verb,143G Minor morphological means of signaling negation
8,5,1,5,4,2,2,2,2,1,1,...,1,3,2,1,2,2,20,2,11,4
34,4,2,4,4,3,4,1,2,1,1,...,2,3,4,2,1,2,19,3,15,4
54,1,2,2,1,1,1,1,1,1,2,...,1,1,1,2,1,4,7,1,1,4
70,2,3,2,2,2,1,1,1,1,1,...,1,1,1,2,6,4,7,1,1,4
89,2,2,3,2,3,1,1,2,1,1,...,2,2,2,2,1,2,19,1,15,4


In [273]:
numerized.describe()

,1A Consonant Inventories,2A Vowel Quality Inventories,3A Consonant-Vowel Ratio,4A Voicing in Plosives and Fricatives,5A Voicing and Gaps in Plosive Systems,6A Uvular Consonants,7A Glottalized Consonants,8A Lateral Consonants,11A Front Rounded Vowels,13A Tone,...,88A Order of Demonstrative and Noun,89A Order of Numeral and Noun,97A Relationship between the Order of Object and Verb and the Order of Adjective and Noun,112A Negative Morphemes,116A Polar Questions,143F Postverbal Negative Morphemes,"144A Position of Negative Word With Respect to Subject, Object, and Verb",143E Preverbal Negative Morphemes,143A Order of Negative Morpheme and Verb,143G Minor morphological means of signaling negation
count,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,...,107.000000,107.000000,107.00000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000
mean,2.719626,2.084112,2.915888,2.345794,1.626168,1.411215,1.504673,2.102804,1.102804,1.420561,...,1.663551,1.401869,2.71028,2.280374,2.121495,2.953271,13.009346,1.990654,4.495327,3.971963
std,1.195861,0.702039,1.182590,1.267167,0.852558,0.961065,1.239095,0.889505,0.474532,0.740186,...,1.302711,0.684874,1.37367,1.465286,1.998633,1.231529,7.239169,1.313898,5.047648,0.290021
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,...,1.000000,1.000000,2.00000,1.000000,1.000000,2.000000,7.000000,1.000000,1.000000,4.000000
50%,3.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,...,1.000000,1.000000,3.00000,2.000000,1.000000,4.000000,16.000000,1.000000,2.000000,4.000000
75%,3.500000,3.000000,4.000000,4.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,...,2.000000,2.000000,4.00000,2.000000,2.000000,4.000000,20.000000,4.000000,4.000000,4.000000
max,5.000000,3.000000,5.000000,4.000000,5.000000,4.000000,6.000000,5.000000,4.000000,3.000000,...,6.000000,4.000000,5.00000,6.000000,7.000000,4.000000,21.000000,4.000000,15.000000,4.000000


top 5 by std:

In [277]:
numerized.std()

1A Consonant Inventories                                                                     1.195861
2A Vowel Quality Inventories                                                                 0.702039
3A Consonant-Vowel Ratio                                                                     1.182590
4A Voicing in Plosives and Fricatives                                                        1.267167
5A Voicing and Gaps in Plosive Systems                                                       0.852558
6A Uvular Consonants                                                                         0.961065
7A Glottalized Consonants                                                                    1.239095
8A Lateral Consonants                                                                        0.889505
11A Front Rounded Vowels                                                                     0.474532
13A Tone                                                                          

In [285]:
most_diverse  = numpy.argsort(numerized.std().values)[:5]
most_diverse_names = complete2raw.columns.take(most_diverse)
most_diverse_names

Index(['143G Minor morphological means of signaling negation',
       '11A Front Rounded Vowels', '82A Order of Subject and Verb',
       '83A Order of Object and Verb', '86A Order of Genitive and Noun'],
      dtype='object')

In [289]:
complete2reduced = complete2raw[most_diverse_names]
complete2reduced

,143G Minor morphological means of signaling negation,11A Front Rounded Vowels,82A Order of Subject and Verb,83A Order of Object and Verb,86A Order of Genitive and Noun
8,4 None,1 None,1 SV,1 OV,1 Genitive-Noun
34,4 None,1 None,1 SV,2 VO,2 Noun-Genitive
54,4 None,1 None,1 SV,1 OV,1 Genitive-Noun
70,4 None,1 None,1 SV,1 OV,1 Genitive-Noun
89,4 None,1 None,1 SV,1 OV,1 Genitive-Noun
128,4 None,1 None,1 SV,3 No dominant order,1 Genitive-Noun
140,4 None,1 None,1 SV,3 No dominant order,1 Genitive-Noun
143,4 None,1 None,1 SV,2 VO,3 No dominant order
153,4 None,1 None,1 SV,1 OV,1 Genitive-Noun
183,4 None,1 None,1 SV,1 OV,1 Genitive-Noun


In [290]:
complete2reduced.to_csv('five-diversex107.csv')